In [14]:
import pandas as pd
import os
import numpy as np


from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
# Specify the directory where the dataset is located
dataset_dir = "dataset_preparation"

In [4]:
# Specify the file name you want to load
file_name = "df_features_with_ranking.csv"  # Change this to the name of your CSV file

# Construct the file path
file_path = os.path.join(dataset_dir, file_name)

# Check if the file exists
if os.path.exists(file_path):
    # Load the CSV file into a DataFrame
    df_features = pd.read_csv(file_path)
    print("df_features csv loaded successfully.")
else:
    print(f"File '{file_name}' does not exist in the directory 'dataset_preparation'.")

DataFrame loaded successfully.


In [9]:
# Specify the file name you want to load
file_name = "predicted_results.csv"  # Change this to the name of your CSV file

# Construct the file path
file_path = os.path.join(dataset_dir, file_name)

# Check if the file exists
if os.path.exists(file_path):
    # Load the CSV file into a DataFrame
    predicted_df_final = pd.read_csv(file_path)
    print("DataFrame loaded successfully.")
else:
    print(f"File '{file_name}' does not exist in the directory 'dataset_preparation'.")

DataFrame loaded successfully.


In [10]:
# Specify the file name you want to load
file_name = "mae_results.csv"  # Change this to the name of your CSV file

# Construct the file path
file_path = os.path.join(dataset_dir, file_name)

# Check if the file exists
if os.path.exists(file_path):
    # Load the CSV file into a DataFrame
    mae_df_final = pd.read_csv(file_path)
    print("DataFrame loaded successfully.")
else:
    print(f"File '{file_name}' does not exist in the directory 'dataset_preparation'.")

DataFrame loaded successfully.


In [5]:
df_features

,unique_id,hurst,series_length,unitroot_pp,unitroot_kpss,hw_alpha,hw_beta,hw_gamma,stability,nperiods,...,crossing_points,arch_lm,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,model_rank
0,realAdExchange/exchange-2_cpc_results.csv,0.906114,1648,-291.651096,3.322436,0.662955,3.689681e-10,2.177951e-10,0.164412,1,...,219,0.333751,0.844560,1.723964,0.019727,0.033048,-0.474200,0.247259,0.533564,"['exponential_smoothing', 'xgboost', 'arima']"
1,realAdExchange/exchange-2_cpm_results.csv,0.720425,1648,-352.407911,0.688890,0.413818,1.154891e-09,1.322620e-09,0.041318,1,...,228,0.270601,0.823062,2.040337,-0.122537,0.066928,-0.590594,0.359987,0.601838,"['exponential_smoothing', 'xgboost', 'arima']"
2,realAdExchange/exchange-3_cpc_results.csv,0.827410,1647,-689.702090,0.784406,0.248606,0.000000e+00,0.000000e+00,0.072987,1,...,361,0.243241,0.595922,0.603031,-0.176902,0.101971,-0.466519,0.270492,0.155811,"['exponential_smoothing', 'xgboost', 'arima']"
3,realAdExchange/exchange-3_cpm_results.csv,0.898996,1647,-377.537596,3.913879,0.716322,6.293205e-10,1.216726e-10,0.143555,1,...,257,0.098898,0.764779,1.248005,-0.058560,0.040519,-0.454639,0.217878,0.290427,"['exponential_smoothing', 'xgboost', 'arima']"
4,realAdExchange/exchange-4_cpc_results.csv,0.520317,1647,-1862.138673,1.727024,0.014715,1.600749e-13,4.137589e-09,0.026716,1,...,287,0.003958,0.040633,0.032792,-0.504576,0.263658,-0.670898,0.492239,-0.000595,"['exponential_smoothing', 'xgboost', 'arima']"
5,realAdExchange/exchange-4_cpm_results.csv,0.390779,1647,-1839.697103,0.181736,0.006275,2.636700e-12,5.935964e-15,0.011183,1,...,315,0.005727,0.030553,0.026503,-0.504210,0.263340,-0.671191,0.494804,0.000060,"['xgboost', 'exponential_smoothing', 'arima']"


In [12]:
predicted_df_final

,dir,file_name,original_value,exponential_smoothing,arima,xgboost
0,realAdExchange,exchange-2_cpc_results.csv,[0.07079632 0.06066116 0.04424014 0.06160637 0...,"[0.08239459853498723, 0.07737155060185466, 0.0...","[0.0739106901218, 0.0739106901218, 0.073910690...","[0.08348382264375687, 0.08224911242723465, 0.0..."
1,realAdExchange,exchange-2_cpm_results.csv,[1.76334284e-01 1.68131390e-01 1.26134291e-01 ...,"[0.2023933033637229, 0.17521874384645642, 0.15...","[0.20361228624551847, 0.2111542908079831, 0.22...","[0.22574101388454437, 0.18855653703212738, 0.1..."
2,realAdExchange,exchange-3_cpc_results.csv,[0.08336749 0.11503169 0.11255595 0.09707841 0...,"[0.08662693547130892, 0.10909349776190366, 0.1...","[0.1012880208061552, 0.11480172722007481, 0.11...","[0.09825707972049713, 0.10299435257911682, 0.1..."
3,realAdExchange,exchange-3_cpm_results.csv,[0.54497814 0.75863932 0.7323819 0.57064705 0...,"[0.6532485519967688, 0.814394912520056, 0.8096...","[0.548446739751, 0.548446739751, 0.54844673975...","[0.576107382774353, 0.576107382774353, 0.56887..."
4,realAdExchange,exchange-4_cpc_results.csv,[0.06864377 0.09565012 0.0599736 0.0620127 0...,"[0.07563152461680721, 0.09054939726141119, 0.0...","[0.08002699251479879, 0.07317117125065187, 0.0...","[0.06404364109039307, 0.06404364109039307, 0.0..."
5,realAdExchange,exchange-4_cpm_results.csv,[ 0.42784913 0.59551545 0.31512822 0.303507...,"[0.4650382897217444, 0.5413448956773631, 0.266...","[0.5017022758048947, 0.4527749809280139, 0.425...","[0.4120841324329376, 0.4120841324329376, 0.405..."


In [11]:
mae_df_final

,dir,file_name,exponential_smoothing,arima,xgboost
0,realAdExchange,exchange-2_cpc_results.csv,0.014086,0.039823,0.014942
1,realAdExchange,exchange-2_cpm_results.csv,0.064484,0.134445,0.067883
2,realAdExchange,exchange-3_cpc_results.csv,0.030153,0.035638,0.030289
3,realAdExchange,exchange-3_cpm_results.csv,0.184865,0.407121,0.274369
4,realAdExchange,exchange-4_cpc_results.csv,0.043871,0.046636,0.046510
5,realAdExchange,exchange-4_cpm_results.csv,0.228934,0.232102,0.228178


In [8]:
# Create an empty column named 'ensemble_size'
df_features['ensemble_size'] = np.nan
# Display the DataFrame with the new empty column
df_features

,unique_id,hurst,series_length,unitroot_pp,unitroot_kpss,hw_alpha,hw_beta,hw_gamma,stability,nperiods,...,arch_lm,x_acf1,x_acf10,diff1_acf1,diff1_acf10,diff2_acf1,diff2_acf10,seas_acf1,model_rank,ensemble_size
0,realAdExchange/exchange-2_cpc_results.csv,0.906114,1648,-291.651096,3.322436,0.662955,3.689681e-10,2.177951e-10,0.164412,1,...,0.333751,0.844560,1.723964,0.019727,0.033048,-0.474200,0.247259,0.533564,"['exponential_smoothing', 'xgboost', 'arima']",NaN
1,realAdExchange/exchange-2_cpm_results.csv,0.720425,1648,-352.407911,0.688890,0.413818,1.154891e-09,1.322620e-09,0.041318,1,...,0.270601,0.823062,2.040337,-0.122537,0.066928,-0.590594,0.359987,0.601838,"['exponential_smoothing', 'xgboost', 'arima']",NaN
2,realAdExchange/exchange-3_cpc_results.csv,0.827410,1647,-689.702090,0.784406,0.248606,0.000000e+00,0.000000e+00,0.072987,1,...,0.243241,0.595922,0.603031,-0.176902,0.101971,-0.466519,0.270492,0.155811,"['exponential_smoothing', 'xgboost', 'arima']",NaN
3,realAdExchange/exchange-3_cpm_results.csv,0.898996,1647,-377.537596,3.913879,0.716322,6.293205e-10,1.216726e-10,0.143555,1,...,0.098898,0.764779,1.248005,-0.058560,0.040519,-0.454639,0.217878,0.290427,"['exponential_smoothing', 'xgboost', 'arima']",NaN
4,realAdExchange/exchange-4_cpc_results.csv,0.520317,1647,-1862.138673,1.727024,0.014715,1.600749e-13,4.137589e-09,0.026716,1,...,0.003958,0.040633,0.032792,-0.504576,0.263658,-0.670898,0.492239,-0.000595,"['exponential_smoothing', 'xgboost', 'arima']",NaN
5,realAdExchange/exchange-4_cpm_results.csv,0.390779,1647,-1839.697103,0.181736,0.006275,2.636700e-12,5.935964e-15,0.011183,1,...,0.005727,0.030553,0.026503,-0.504210,0.263340,-0.671191,0.494804,0.000060,"['xgboost', 'exponential_smoothing', 'arima']",NaN


In [16]:
def stacked_model_predictions(val, base_preds):
    # Splitting features and target variable
    X_train, X_val, y_train, y_val = train_test_split(base_preds, val, test_size=0.2, random_state=42)

    # Define parameter grid for Random Forest
    param_grid = {
        'n_estimators': [25, 50, 100, 150, 200],  # Number of trees in the forest
        'max_depth': [None, 10, 20, 30],      # Maximum depth of the tree
        'min_samples_split': [2, 5, 8, 10, 15],  # Minimum number of samples required to split an internal node
        'min_samples_leaf': [1, 2, 4, 6]     # Minimum number of samples required to be at a leaf node
    }

    # Initialize Random Forest regressor
    rf = RandomForestRegressor(random_state=42)

    search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=100, cv=2, scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42)
    search.fit(X_train, y_train)

    print("Stacking Approach")

    # Print the best estimator found
    print(search.best_estimator_)

    # Make predictions using the best model
    y_pred = search.best_estimator_.predict(X_val)

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_val, y_pred)
    print("Mean Absolute Error (MAE):", mae)

    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(y_val, y_pred)
    print("Mean Squared Error (MSE):", mse)

    # Calculate Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)
    print("Root Mean Squared Error (RMSE):", rmse)

    print("")

    return y_pred, y_val, mae, mse

In [41]:
def find_ensemble_size(model_rank, unique_id):
    # Extract directory and file name from unique_id
    dir_name, file_name = unique_id.split('/')

    print(model_rank)
    # Get the first element from the list of model_rank
    #model_name = model_rank[0]
    model_name = model_rank.strip("[]").split(',')[0].strip("' ")
    print("Model Name:", model_name)

    # Find the row in mae_df dataframe that matches the directory and file name
    row = mae_df_final[(mae_df_final['dir'] == dir_name) & (mae_df_final['file_name'] == file_name)]

    # Find the value in the column that matches the model_name
    model_mae = row[model_name].iloc[0]

    # Determine the ensemble size based on the model MAE value
    if model_mae < 0.05:
        print("No need for stacking apporach, since first model has MAE less than 0.05")
        return 1
    else:
        previous_mae = model_mae
        print("Going for stacking Approach")
        i = 0  # Initialize the count of models
        while i < len(model_rank):
            i += 2  # Increment the count of models

            # Take the first i models from the model_rank list
            models_to_use = model_rank.strip("[]").split(',')[:i]
            base_preds = []  # Initialize base_preds as a list
            # Get the predicted values for the selected models
            for model in models_to_use:
                # Find the respective row in predicted_df
                model_row = predicted_df_final[(predicted_df_final['dir'] == dir_name) & (predicted_df_final['file_name'] == file_name)]
                # Get the predicted value for the model
                pred_value = model_row[model.strip("' ")].iloc[0]

                # Append the predicted value to base_preds
                base_preds.append(pred_value)

            val_row = predicted_df_final[(predicted_df_final['dir'] == dir_name) & (predicted_df_final['file_name'] == file_name)]
            val = val_row['original_value'].iloc[0]

            base_preds = np.stack(base_preds, axis=-1)
            y_pred, y_val, mae, mse = stacked_model_predictions(val, base_preds)

            if mae > previous_mae:
                print("MAE increased after adding", i, "models, so returning the previous ensemble size")
                return i - 1

            # Update previous MAE with current MAE
            previous_mae = mae

            # If MAE is less than 0.05, return the current ensemble size
            if mae < 0.08:
                return i

    # If none of the models have MAE less than 0.05, return the total count of models
    return len(model_rank)

In [ ]:
# Iterate over rows in df_features
for index, row in df_features.iterrows():

    # Extract values from the current row
    model_rank = row['model_rank']
    unique_id = row['unique_id']

    # Print the unique ID before calling the function
    print("Unique ID:", unique_id)

    # Call the function to find ensemble size
    ensemble_size = find_ensemble_size(model_rank, unique_id)

    print("")

    # Assign the ensemble size to the 'ensemble_size' column
    df_features.at[index, 'ensemble_size'] = ensemble_size